# Project 1: Customer Database
**This is the first of three mandatory projects to be handed in as part of the assessment for the course 02807 Computational Tools for Data Science at Technical University of Denmark, autumn 2019.**

#### Practical info
- **The project is to be done in groups of at most 3 students**
- **Each group has to hand in _one_ Jupyter notebook (this notebook) with their solution**
- **The hand-in of the notebook is due 2019-10-13, 23:59 on DTU Inside**

#### Your solution
- **Your solution should be in Python**
- **For each question you may use as many cells for your solution as you like**
- **You should document your solution and explain the choices you've made (for example by using multiple cells and use Markdown to assist the reader of the notebook)**
- **You should not remove the problem statements, and you should not modify the structure of the notebook**
- **Your notebook should be runnable, i.e., clicking [>>] in Jupyter should generate the result that you want to be assessed**
- **You are not expected to use machine learning to solve any of the exercises**
- **You will be assessed according to correctness and readability of your code, choice of solution, choice of tools and libraries, and documentation of your solution**

## Introduction
Your team has been hired by the company X as data scientists. X makes gadgets for a wide range of industrial and commercial clients.

As in-house data scientists, your teams first task, as per request from your new boss, is to optimize business operations. You have decided that a good first step would be to analyze the companys historical sales data to gain a better understanding of where profit is coming from. It may also reveal some low hanging fruit in terms of business opportunities.

To get started, you have called the IT department to get access to the customer and sales transactions database. To your horror you've been told that such a database doens't exist, and the only record of sales transactions is kept by John from finance in an Excel spreadsheet. So you've emailed John asking for a CSV dump of the spreadsheet...

In this project you need to clean the data you got from John, enrich it with further data, prepare a database for the data, and do some data analysis. The project is comprised of five parts. They are intended to be solved in the order they appear, but it is highly recommended that you read through all of them and devise an overall strategy before you start implementing anything.

## Part 1: Cleaning the data
John has emailed you the following link to the CSV dump you requested.

- [transactions.csv](https://raw.githubusercontent.com/patrickcording/02807-comp-tools/master/docker/work/data/transactions.csv)

It seems as though he has been a bit sloppy when keeping the records. 

In this part you should:
- Explain what the data is
- Clean it to prepare it for inserting into a database and doing data analysis 

---
### Solution

In the CSV file given to us by John from finances, a large table can be found. 

Each row represents a transaction from a sale of gadgets to a company. 

The file is loaded to get more information.

In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('transactions.csv', encoding='utf-8-sig')
df.head(3)

,part,company,country,city,price,date
0,54868-5165,Chatterbridge,Spain,Barcelona,784.79€,2016-01-02 00:01:05
1,60505-2867,Lajo,Greece,Thessaloniki,187.99€,2016-01-02 00:05:26
2,24385-268,Flipstorm,Greece,Athens,221.73€,2016-01-02 00:18:30


In [3]:
len(df)

20568

The table consists of 20568 data rows and there are 6 columns of information for each transaction. 

The first column, 'part', is not unique nor does it give information important in regards to doing this exercise. Therefore, it is removed from the table.

In [4]:
df.drop(['part'], inplace=True, axis=1)
df.head(3)

,company,country,city,price,date
0,Chatterbridge,Spain,Barcelona,784.79€,2016-01-02 00:01:05
1,Lajo,Greece,Thessaloniki,187.99€,2016-01-02 00:05:26
2,Flipstorm,Greece,Athens,221.73€,2016-01-02 00:18:30


To gain a better knowledge of the type of data, the first order of business is to have python tell us what type of data we are working with. 

In [5]:
df.dtypes

company    object
country    object
city       object
price      object
date       object
dtype: object

The above output tells us that the data in all the columns is of 'object' type - also known as 'string' type. However, from the table it is clear that the data in the 'price' and 'date' columns should not be an 'object' but rather a 'float' and 'datetime', respectively. 

Now when we know what the data is, we need to make a plan for how to clean it:

- Make sure that all dates in the 'date' column are real dates and in the same format.
- Remove all the NaN values from company, country and city.
- Clean the 'price' column by removing rows without a price and make sure all currencies are in euro. 
- Remove duplicates.

#### Make sure that all dates in the 'date' column are real dates and in the same format.

***I:*** We find the transactions with the default date format. If a transaction is not in the default format it is changed to it. Further, if a date is a non-valid date it is removed (e.g. 2017-13-32). 

In [6]:
out_of_range = []

for i in range(0,len(df)):
    try:
        pd.to_datetime(df.iloc[i,4], format='%Y-%m-%d')
    except:
        try: 
            df.iloc[i,4] = pd.to_datetime(df.iloc[i,4])
        except:
            out_of_range.append(i)

df.drop(out_of_range, inplace=True)

***II:*** We change the dtype from object to datetime64. 

In [7]:
df['date'] = df['date'].astype('datetime64[ns]')
df.dtypes

company            object
country            object
city               object
price              object
date       datetime64[ns]
dtype: object

***SUCCES!*** All dates are now changed to the same format and the dtype is changed to datetime64.

#### Remove all the NaN values from company, country and city.

***I:*** We check which companies are unique in which countries.

In [8]:
grouped = df.groupby('company')['country'].unique().apply(list).to_dict()
print(grouped)

{' -': ['United States'], ' a': ['United States'], 'Avaveo': ['France', nan], 'Brainsphere': ['Portugal', nan, 'Portuga'], 'Bubblemix': ['Japan', nan], 'Buzzbean': ['Germany', nan, 'Tyskland'], 'Chatterbridge': ['Spain', nan], 'Eimbee': ['France', nan], 'Flipstorm': ['Greece', 'France', nan], 'Gabcube': ['Portugal', nan], 'Gabtune': ['France', nan], 'Gevee': ['France', nan], 'Innojam': ['Netherlands', nan], 'Kanoodle': ['Japan', nan], 'Laj0': ['Greece'], 'Lajo': ['Greece', nan], 'Ntags': ['Portugal', nan, 'Portuga'], 'Ntagz': ['Portugal'], 'Realpoint': ['Portugal', nan], 'Rhycero': ['France', nan], 'Riffpath': ['Greece', nan], 'Roodel': ['Portugal', nan], 'Shufflebeat': ['Portugal', nan], 'Tagtune': ['Switzerland', nan], 'Teklist': ['Netherlands', nan], 'Thoughtmix': ['Portugal', nan, 'Portuga'], 'Thoughtmixz': ['Portugal'], 'Twitterbeat': ['France', nan], 'Voomm': ['France', nan], 'Wordify': ['United States', nan], 'Yozio': ['Greece', nan], 'Zoonder': ['United States', 'US', nan], 'Zo

***II:*** We see that some companies and countries are close to identical and we figure that these must be typos. These are corrected.

In [9]:
for i in range(0,len(df)):
    if df.iloc[i,0] == 'Zooxo.':
        df.iloc[i,0] = 'Zooxo'
    elif df.iloc[i,0] == 'Thoughtmixz':
        df.iloc[i,0] = 'Thoughtmix'
    elif df.iloc[i,0] == 'Ntagz':
        df.iloc[i,0] = 'Ntags'
    elif df.iloc[i,0] == 'Laj0':
        df.iloc[i,0] = 'Lajo'
    elif df.iloc[i,1]=='US':
        df.iloc[i,1] = 'United States'
    elif df.iloc[i,1]=='Tyskland':
        df.iloc[i,1] = 'Germany'
    elif df.iloc[i,1]=='Portuga':
        df.iloc[i,1] = 'Portugal'  

In [10]:
grouped = df.groupby('company')['country'].unique().apply(list).to_dict()

***III:*** Most of the companies now only have one country and a NaN value in the 'country' column. Therefore, we make the assumption that in these cases the company is only located in one specific country. All the NaN value are switched to this specific country.

In [11]:
for i in range(0,len(df)):
    if pd.isna(df.iloc[i,1]) == True:
        if len(grouped[df.iloc[i,0]]) == 2:
            df.iloc[i,1] = grouped[df.iloc[i,0]][0]

***SUCCES!*** Most companies (except 'Flipstorm') now only have one country and no NaN value should be present.

In [12]:
grouped = df.groupby('company')['country'].unique().apply(list).to_dict()
grouped

{' -': ['United States'],
 ' a': ['United States'],
 'Avaveo': ['France'],
 'Brainsphere': ['Portugal'],
 'Bubblemix': ['Japan'],
 'Buzzbean': ['Germany'],
 'Chatterbridge': ['Spain'],
 'Eimbee': ['France'],
 'Flipstorm': ['Greece', 'France', nan],
 'Gabcube': ['Portugal'],
 'Gabtune': ['France'],
 'Gevee': ['France'],
 'Innojam': ['Netherlands'],
 'Kanoodle': ['Japan'],
 'Lajo': ['Greece'],
 'Ntags': ['Portugal'],
 'Realpoint': ['Portugal'],
 'Rhycero': ['France'],
 'Riffpath': ['Greece'],
 'Roodel': ['Portugal'],
 'Shufflebeat': ['Portugal'],
 'Tagtune': ['Switzerland'],
 'Teklist': ['Netherlands'],
 'Thoughtmix': ['Portugal'],
 'Twitterbeat': ['France'],
 'Voomm': ['France'],
 'Wordify': ['United States'],
 'Yozio': ['Greece'],
 'Zoonder': ['United States'],
 'Zooxo': ['United Kingdom'],
 'aa': ['United States']}

***IV:*** We now check which companies are unique in which countries AND cities.

In [13]:
grouped = (df.groupby(['company'])['country', 'city']
       .apply(lambda x: pd.unique(x.values.ravel()).tolist()))
grouped

company
 -                               [United States, Boston]
 a                             [United States, New York]
Avaveo                                    [France, Nice]
Brainsphere               [Portugal, Braga, nan, Monção]
Bubblemix                                 [Japan, Asaka]
Buzzbean                           [Germany, Düsseldorf]
Chatterbridge                         [Spain, Barcelona]
Eimbee                                  [France, Amiens]
Flipstorm        [Greece, Athens, France, Nanterre, nan]
Gabcube                               [Portugal, Almada]
Gabtune                                   [France, Lyon]
Gevee                              [France, Champagnole]
Innojam                         [Netherlands, Amsterdam]
Kanoodle                           [Japan, Niihama, nan]
Lajo                              [Greece, Thessaloniki]
Ntags                            [Portugal, Lisbon, nan]
Realpoint                             [Portugal, Lisbon]
Rhycero                

We see that the dream scenario of having a company in only one country and one city is far from fulfilled. 

***V:*** Three companies, 'aa', ' a' and ' -', seem suspicious. For each of these companies it is clear that only one other company has the exact same location. Tus, we assume that the three suspicious companies are actually typos of the companies with the same location.

In [14]:
for i in range(0,len(df)):
    if df.iloc[i,0] == 'aa':
        df.iloc[i,0] = 'Wordify'
    elif df.iloc[i,0] == ' a':
        df.iloc[i,0] = 'Wordify'
    elif df.iloc[i,0] == ' -':
        df.iloc[i,0] = 'Zoonder'

***VI:*** We see that the company 'Brainsphere' is in two different cities in Portugal. So we count how many times the company occurs in each city.

In [15]:
count_braga = count_moncao = 0

for i in range(0,len(df)):
    if df.iloc[i,2] == 'Braga':
        count_braga += 1
    elif df.iloc[i,2] == 'Monção':
        count_moncao += 1
        
print('Braga: %s\nMonção: %s' % (count_braga, count_moncao))

Braga: 1237
Monção: 1


***VII:*** It is now clear that John from finances made a mistake since 'Brainspehere' is only found once in Monção. So we change the city from Monção to Braga.

In [16]:
for i in range(0,len(df)):
    if df.iloc[i,2] == 'Monção':
        df.iloc[i,2] = 'Braga'

***VIII:*** We see that the company 'Thoughtmix' is in two different cities in Portugal. But after a quick google search we realize that John from finances once again made a mistake. Vila Fria is not a city but an area in Amadora and thus we changed it to Amadora. Note: We also removed the extra /t from Amadora. 

In [17]:
for i in range(0,len(df)):
    if df.iloc[i,2] == 'Amadora\t':
        df.iloc[i,2] = 'Amadora'
    elif df.iloc[i,2] == 'Vila Fria':
        df.iloc[i,2] = 'Amadora'

In [18]:
grouped = (df.groupby(['company'])['country', 'city']
       .apply(lambda x: pd.unique(x.values.ravel()).tolist()))
grouped

company
Avaveo                                    [France, Nice]
Brainsphere                       [Portugal, Braga, nan]
Bubblemix                                 [Japan, Asaka]
Buzzbean                           [Germany, Düsseldorf]
Chatterbridge                         [Spain, Barcelona]
Eimbee                                  [France, Amiens]
Flipstorm        [Greece, Athens, France, Nanterre, nan]
Gabcube                               [Portugal, Almada]
Gabtune                                   [France, Lyon]
Gevee                              [France, Champagnole]
Innojam                         [Netherlands, Amsterdam]
Kanoodle                           [Japan, Niihama, nan]
Lajo                              [Greece, Thessaloniki]
Ntags                            [Portugal, Lisbon, nan]
Realpoint                             [Portugal, Lisbon]
Rhycero                                [France, Arcueil]
Riffpath                             [Greece, Heraklion]
Roodel                 

***IX:*** Most of the companies now only have one specific country, one specific city and a NaN value in the 'city' column (except the company 'Flipstorm'). All the NaN values are switched to the specific city.

In [19]:
for i in range(0,len(df)):
    if pd.isna(df.iloc[i,2]) == True:
        if len(grouped[df.iloc[i,0]]) == 3:
            df.iloc[i,2] = grouped[df.iloc[i,0]][1]

***X:*** Now only the company 'Flipstorm' is messed up. This is easiely fixed. If the country is Greece the city should be Athens, and vice versa. If the country is France the city should be Nanterre, and vice versa. 

In [20]:
 for i in range(0,len(df)):
    if df.iloc[i,1] == 'Greece':
        df.iloc[i,2] = 'Athens'
    elif df.iloc[i,1] == 'France':
        df.iloc[i,2] = 'Nanterre'
    elif df.iloc[i,2] == 'Athens':
        df.iloc[i,1] = 'Greece'
    elif df.iloc[i,2] == 'Nanterre':
        df.iloc[i,1] = 'France'

***XI:*** We do a final check.

In [21]:
grouped = (df.groupby(['company'])['country', 'city']
       .apply(lambda x: pd.unique(x.values.ravel()).tolist()))
grouped

company
Avaveo                           [France, Nanterre]
Brainsphere                       [Portugal, Braga]
Bubblemix                            [Japan, Asaka]
Buzzbean                      [Germany, Düsseldorf]
Chatterbridge                    [Spain, Barcelona]
Eimbee                           [France, Nanterre]
Flipstorm        [Greece, Athens, France, Nanterre]
Gabcube                          [Portugal, Almada]
Gabtune                          [France, Nanterre]
Gevee                            [France, Nanterre]
Innojam                    [Netherlands, Amsterdam]
Kanoodle                           [Japan, Niihama]
Lajo                               [Greece, Athens]
Ntags                            [Portugal, Lisbon]
Realpoint                        [Portugal, Lisbon]
Rhycero                          [France, Nanterre]
Riffpath                           [Greece, Athens]
Roodel                          [Portugal, Aranhas]
Shufflebeat                       [Portugal, Porto]
Tagt

***SUCCES!*** Now there are no more NaN values.

#### Clean the 'price' column by removing rows without a price and make sure all currencies are in euro.

***I:*** We find all the unique entries in the price column that are not a price in euro.

In [22]:
price = df.iloc[:,3].astype(str)

listunique = []
for i in range(0,len(df)):
    price = str(df.iloc[i,3])
    if '€' not in price:
        regex1 = re.search(r'^(\D+)[-]?\d*[.]*\d*', price)
        regex2 = re.search(r'[-]?\d*[.]*\d*(\D+)$', price)
        if regex1 is not None:
            group1 = regex1.group(1)
            listunique.append(group1)
        elif regex2 is not None:
            group2 = regex2.group(1)
            listunique.append(group2)

onlylistunique = []
for elem in listunique:
    if elem not in onlylistunique:
        onlylistunique.append(elem)   
print(onlylistunique)

['£', '$', '¥', '£-', '¥-', '$-', '-', 'void', 'nan', 'na']


***II:*** We change the currency to euro in lines with '£', '$' and '¥' and for all the prices the currency symbol is removed.

In [23]:
for i in range(0,len(df)):  
    price = str(df.iloc[i,3])
    if '€' not in price:
        if '£' in price:
            pricepound = float(df.iloc[i,3][1:]) * 1.12305
            df.iloc[i,3] = round(pricepound,2)
        elif '$' in price:
            pricedollar = float(df.iloc[i,3][1:]) * 0.912104 
            df.iloc[i,3] = round(pricedollar,2)
        elif '¥' in price:
            priceyen = float(df.iloc[i,3][1:]) * 0.00851916
            df.iloc[i,3] = round(priceyen,2)
    else: 
        priceeuro = float(df.iloc[i,3][:-1])
        df.iloc[i,3] = round(priceeuro,2)

df.head(3)

,company,country,city,price,date
0,Chatterbridge,Spain,Barcelona,784.79,2016-01-02 00:01:05
1,Lajo,Greece,Athens,187.99,2016-01-02 00:05:26
2,Flipstorm,Greece,Athens,221.73,2016-01-02 00:18:30


Websites used for currency:

- Currency for £ to €: https://www.xe.com/currencyconverter/convert/?Amount=1&From=GBP&To=EUR visited: 3/10
- Currency for $ to €: https://www.xe.com/currencyconverter/convert/?Amount=1&From=USD&To=EUR visited: 3/10
- Currency for ¥ to €: https://www.xe.com/currencyconverter/convert/?Amount=1&From=JPY&To=EUR visited: 3/10

***III***: We delete the lines with 'void', 'na', 'nan' and '-'

In [24]:
remove_price = []

for index, row in df.iterrows():
    if row['price'] == 'void' or row['price'] == 'na' or row['price'] == 'nan' or row['price'] == '-':
        remove_price.append(index)  
        
df.drop(remove_price, inplace = True)

***IV***: The data in the 'price' column is changed to float64.

In [25]:
df['price'] = pd.to_numeric(df['price'])
df.dtypes

company            object
country            object
city               object
price             float64
date       datetime64[ns]
dtype: object

***SUCCESS!*** The currencies have been changed to euro and the data type is now float. 

#### Remove duplicates.

In [26]:
df = df.drop_duplicates()

#### Status.

The data has been cleaned and is now ready for further analysis. 

In [27]:
len(df)

20556

---
## Part 2: Enriching the data

A common task for a data scientists is to combine or enrich data from internal sources with data available from external sources. The purpose of this can be either to fix issues with the data or to make it easier to derive insights from the data.

In this part you should enrich your data with data from at least one external source. You may look to part 4 for some  inspiration as to what is required. Your solution should be automated, i.e., you can not ask the reader of your notebook to download any data manually. You should argue why and what you expect to achieve by the enrichments you are doing.

---
### Solution

In [28]:
# Loading the file new file.
new_df = pd.read_csv('worldbank_info.csv', encoding='utf-8-sig')

In [29]:
new_df.head(5)

,country,gdp_euro,gini_index
0,France,35367.24443,32.3
1,Germany,40876.16230,31.4
2,Greece,17109.21725,35.8
3,Japan,35322.60749,32.1
4,Netherlands,44326.09763,28.6


In [30]:
df = pd.merge(df, new_df, on='country',how='inner')

In [31]:
df.head(5)

,company,country,city,price,date,gdp_euro,gini_index
0,Lajo,Greece,Athens,187.99,2016-01-02 00:05:26,17109.21725,35.8
1,Flipstorm,Greece,Athens,221.73,2016-01-02 00:18:30,17109.21725,35.8
2,Lajo,Greece,Athens,317.65,2016-01-02 11:01:32,17109.21725,35.8
3,Flipstorm,Greece,Athens,841.97,2016-01-02 12:51:30,17109.21725,35.8
4,Flipstorm,Greece,Athens,978.51,2016-01-02 17:45:54,17109.21725,35.8


In [32]:
df.dtypes

company               object
country               object
city                  object
price                float64
date          datetime64[ns]
gdp_euro             float64
gini_index           float64
dtype: object

In [33]:
len(df)

18969

---
## Part 3: Creating a database
Storing data in a relational database has the advantages that it is persistent, fast to query, and it will be easier access for other employees at Weyland-Yutani.

In this part you should:
- Create a database and table(s) for the data
- Insert data into the tables

You may use SQLite locally to do this. You should argue why you choose to store your data the way you do. 

---
### Solution

In [34]:
import sqlite3

In [35]:
conn = sqlite3.connect('transactions.sqlite')

In [36]:
c = conn.cursor()

In [37]:
df.to_sql("transactions", conn, if_exists='replace')

In [38]:
conn.commit()

In [39]:
c.execute('SELECT date(date) FROM transactions')

In [40]:
len(c.fetchall())

18969

---
## Part 4: Analyzing the data
You are now ready to analyze the data. Your goal is to gain some actionable business insights to present to your boss. 

In this part, you should ask some questions and try to answer them based on the data. You should write SQL queries to retrieve the data. For each question, you should state why it is relevant and what you expect to find.

To get you started, you should prepare answers to the following questions. You should add more questions.
#### Who are the most profitable clients?
Knowing which clients that generate the most revenue for the company will assist your boss in distributing customer service ressources.

#### Are there any clients for which profit is declining?
Declining profit from a specific client may indicate that the client is disatisfied with the product. Gaining a new client is often much more work than retaining one. Early warnings about declining profit may help your boss fighting customer churn.


Remember, you are taking this to your new boss, so think about how you present the data.

---
### Solution

---
## Part 5: Performance
Your boss is very impressed with what you have achieved in less than two weeks, and he would like to take your idea of storing the customer and sales data in a relational database to production. However, John is concerned that the solution will not scale. His experience is telling him that you will see many occurrences of the following queries.

- Show all sales to company X between time $t_1$ and time $t_2$
- Show the latest X sales in the database
- Show total sales per company per day

Show that Johns concern is not justified.

---
### Solution

---